<p>Every calculation in vanilla python adhere to IEEE 754 standard, therefore precision in floating point calculations need to be adjusted for each method</p>
    
<h3>In my implementation I use fun(x,y) to define an objective function</h3>

<h3>For each method I created a counter to track the number of iterations each method goes through, and also created a variable to track time it took for one iteration</h3>


In [105]:
#some imports:
from inspect import signature
import random
import numpy as np
import itertools


<h3>I define a function below with x and y</h3>

In [106]:
def fun(x,y):
    return ((x**2+y-11)**2+(x+y**2-7)**2)

<h3>To implement both of the Random jump method and a grid search method I define a function set_boundaries 
    that takes the number of variables to set boundaries to and checks that two numbers are given 
    (I don't check for alphabetical and other because I don't think it is crucial to the task)
<br> [also worth mentioning that I used signature parameters to understand how much arguments a funciton has so that we can scale
    this code for different functions]
</h3>

<h3>To implement a random jump method I set boundaries for each variable, then for each iteration that we wish to proceed with, I find the value of a function at the points that are calculated as follows: l + (r*(u-l)), where l,u are boundaries and r is a random number </h3>

<h3>python's random.random() already generates from 0 to 1 so I used that</h3>
<h3>for each variable I calculate its value and add it to the variable array, then I find the value of function with that given function and add it to the dictionary</h3>
<h3>By finding the lowest value of a dictionary I get my result</h3>
<p>There is probably a better way to store the values by comparison, but I couldn't think of it and because our generated values are random we don't know when we will get to the optimum solution</p>
<p>Number of iterations is given in the fucntion as an argument, and therefore we know how many calculation loops is done, however it is almost never gonna be on point in accuracy because of the randomness </p>

In [109]:
def set_boundaries(n):
    #for n variables we set boundaries
    boundary_array = []
    for i in range(n):
        
        boundary = input(f'''Input boundaries for variable number {i}, (x_{i})
        Please input both numbers separated by comma in this form: l,u ''')
        boundary = tuple(int(x) for x in boundary.split(","))
        while len(boundary) != 2:
            boundary = input(f'''YOU DID NOT GIVE 2 NUMBERS AS BOUNDARIES, PLEASE DO IT AGAIN
            Input boundaries for variable number {i}, (x_{i})''')
            boundary = tuple(int(x) for x in boundary.split(","))
            
        boundary_array.append(boundary)
        
    return boundary_array
        
def random_jump(iterations):
    sig = signature(fun)
    n = (len(sig.parameters)) # to get the number of variables of our objective function
    print("NUMBER OF ARGUMENTS: ", n)
    boundary_array = set_boundaries(n)

    smartly_violated = {}
    
    for y in range(iterations):
        
        variable_array = []
        for i in range(n):
            r = random.random()
            #print("Random number: ", r)
            l, u = boundary_array[i]
            x = l + (r*(u-l))
            #print(f"this number is put as variable (x{i}):",x)
            variable_array.append(x)
            
        #print(variable_array)

        #create a key value pair where key is function value
        #and value is variables of the function that give that function value
        
        smartly_violated[fun(*variable_array)]=variable_array
        
        
        # print(variable_array)
        # print(fun(*variable_array))
        
        answer = min(smartly_violated.keys())
    
    return answer, smartly_violated[answer]



<p>As we can see after 1000 iterations the minimum was found at some points close to 3 and 2 giving a function value
close to 0, Analytically we know the minimum is approached at 0 because we deal with the sum of squares and from expression, the values of x and y should be 3 and 2, and with random jump method we got somewhat close after 1000(!) iterations</p>

In [111]:
random_jump(1000)

NUMBER OF ARGUMENTS:  2


Input boundaries for variable number 0, (x_0)
        Please input both numbers separated by comma in this form: l,u  0,10
Input boundaries for variable number 1, (x_1)
        Please input both numbers separated by comma in this form: l,u  0,10


(0.5036654697776486, [3.0594368086030235, 2.1139266174687235])

<h3>For the implementation of a grid search we once again set boundaries using our set_boundaries(), then with the given
p I can find a proxy (distance between equally spaced points of a grid) to construct a grid </h3>
<h3>I create a grid and using a itertools.product I get all the combinations between my variable arrays in order to recreate
a combination of each points in the grid</h3>
<h3>Then I simply find the values for each combination and put it in a dictionary in the similar manner to the random jump storing method</h3>

<h3>As we can see we go through all of the combinations of a grid, therefore I print out the number of combinations we get</h3>

In [112]:
def grid_search(p):
    sig = signature(fun)
    n = (len(sig.parameters))
    print("NUMBER OF ARGUMENTS: ", n)
    boundary_array = set_boundaries(n)
    
    variables_array = []
    all_iterations = {}
    
    for i in range(n):
        variable_array = []
        l, u = boundary_array[i]
        proxy = abs(l-u)/p
        for _ in range(p+1):
            variable_array.append(l)
            l += proxy
        variables_array.append(variable_array)
        
    print("Grid is set up like this: ",variables_array)
    combinations = list(itertools.product(*variables_array))
    
    # print("all combinations: ", combinations)
    for i in range(len(combinations)):
        all_iterations[fun(*combinations[i])]=combinations[i]
    grid_answer = min(all_iterations.keys())
    
    print("Number of iterations: ",len(combinations))
    # print(all_iterations)
    return grid_answer, all_iterations[grid_answer]

<p>After 121 iterations and a separation by 1 unit ranging from 0 to 10 we can get exactly our answer, however
it might be because I biased it to have integers, but anyway we can guarantee some accuracy because we can create a very tight
grid considering given some limitation boundaries</p>

In [113]:
grid_search(10)

NUMBER OF ARGUMENTS:  2


Input boundaries for variable number 0, (x_0)
        Please input both numbers separated by comma in this form: l,u  0,10
Input boundaries for variable number 1, (x_1)
        Please input both numbers separated by comma in this form: l,u  0,10


Grid is set up like this:  [[0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0], [0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]]
Number of iterations:  121


(0.0, (3.0, 2.0))

<h3>For the last task I just redefine the function fun() and because all the code was already implemented
with scale in mind we can see the results:</h3>

In [114]:
def fun(x1,x2,x3,x4):
    return (x1 - 3)**2 + (x2 + 2)**2 + (x3 * 4)**2 + (x4 // 2)**2

In [115]:
random_jump(1000)

NUMBER OF ARGUMENTS:  4


Input boundaries for variable number 0, (x_0)
        Please input both numbers separated by comma in this form: l,u  -5,5
Input boundaries for variable number 1, (x_1)
        Please input both numbers separated by comma in this form: l,u  -5,5
Input boundaries for variable number 2, (x_2)
        Please input both numbers separated by comma in this form: l,u  -5,5
Input boundaries for variable number 3, (x_3)
        Please input both numbers separated by comma in this form: l,u  -5,5


(2.2458524731965195,
 [1.9428743559167625,
  -2.341071014536472,
  -0.02739572145105562,
  -0.5749574545714573])

In [116]:
grid_search(10)

NUMBER OF ARGUMENTS:  4


Input boundaries for variable number 0, (x_0)
        Please input both numbers separated by comma in this form: l,u  -5,5
Input boundaries for variable number 1, (x_1)
        Please input both numbers separated by comma in this form: l,u  -5,5
Input boundaries for variable number 2, (x_2)
        Please input both numbers separated by comma in this form: l,u  -5,5
Input boundaries for variable number 3, (x_3)
        Please input both numbers separated by comma in this form: l,u  -5,5


Grid is set up like this:  [[-5, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [-5, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [-5, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [-5, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0]]
Number of iterations:  14641


(0.0, (3.0, -2.0, 0.0, 1.0))

<p>In short we can see that after 1000 iterations our best guess at a new function 
[which is obviously 0 at (3,-2,0,1)] is 2.333 with all values almost close to what it should be</p>

<p>In the contrary, the grid search needed 14631 iterations to get spot on the points. Again, I biased it to have a neat grid
but who cares, if it was not integers, it would highly depend on the design of the grid</p>